Authors: Kevin Tan and Xinwen Wang

Adapted from https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

### Neural Machine Translation Using Sequence to Sequence Network with Attention and Radical Enhancement

In this project we will be teaching a neural network to translate from
Chinese to English. Specifically, we will use a sequence-to-sequence endoder 
decoder model with attention, integrating Chinese radicals for improved
machine translation. 


In [0]:
from google.colab import drive
drive.mount('/content/drive')



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd drive/

/content/drive


In [0]:

%cd My Drive

/content/drive/My Drive


In [0]:
%cd LIGN167FinalProject

/content/drive/My Drive/LIGN167FinalProject


In [0]:
%ls

'Copy of Copy of NMT_ChineseEnglish_Seq2Seq_with_Attention_and_Radical.ipynb'
 dataset/
 models/
'NMT_ChineseEnglish_Seq2Seq_with_Attention_and_Radical (1).ipynb'
 NMT_ChineseEnglish_Seq2Seq_with_Attention_and_Radical.ipynb
 notes/
 papers/
'Radical Enhanced Seq2Seq with Attention.ipynb'
 wordseg/
'write up.gdoc'


In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
%matplotlib inline

from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [0]:
torch.cuda.get_device_name(0)

'Tesla K80'

Loading data files
==================

The data for this project is a set of many thousands of Chinese to English translation pairs.





In [0]:

ch_corpus_dir = 'dataset/casia2015/casia2015_ch_small.txt'
en_corpus_dir = 'dataset/casia2015/casia2015_en_small.txt'

We will be representating each word in the Chinese language as a word embedding
using word2vec. In Chinese, words can be formed from a varying number of characters, so
we need to do some preprocessing to segment the corpus into separate words. To do this,
we use https://github.com/Moonshile/ChineseWordSegmentation



In [0]:
import pandas as pd
path='dataset/radical_look_up/xinhua.csv'
radical_set=pd.read_csv(path, header=None)
chars = radical_set[0]
rads = radical_set[1]
radical_dict = dict(zip(chars, rads))


def find_radical(character):
    if radical_dict.get(character) is None:
        radical_dict[character] = ' '
    return radical_dict[character]




We'll need a unique index per word to use as the inputs and targets of
the networks later. To keep track of all this we will use a helper class
called ``Lang`` which has word → index (``word2index``) and index → word
(``index2word``) dictionaries, as well as a count of each word
``word2count`` to use to later replace rare words.




In [0]:
SOS_token = 0
EOS_token = 1
# Chinese word segmentation 
# https://github.com/Moonshile/ChineseWordSegmentation
from wordseg import WordSegment

class ch_Lang:
    def __init__(self, name):
        self.name = name
        self.data_dir = ch_corpus_dir
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS
        with open(self.data_dir, 'r') as myfile:
            data=myfile.read().replace('\n', '')
        self.ws = WordSegment(data, max_word_len=3, min_aggregation=10, min_entropy=0.5)

    def addSentence(self, sentence):
        # Chinese word segmentation
        segmented = self.ws.segSentence(sentence)
        for word in segmented:
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
    
    def getword(self):
      return self.word2index

In [0]:
SOS_token = 0
EOS_token = 1

class en_Lang:
  
    def __init__(self, name):
        self.name = name
        self.data_dir = en_corpus_dir
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [0]:
SOS_token = 0
EOS_token = 1

# Chinese word segmentation 
# https://github.com/Moonshile/ChineseWordSegmentation
from wordseg import WordSegment

class radical_Lang:
    def __init__(self, name):
      self.name = name
      self.data_dir = ch_corpus_dir
      self.word2index = {}
      self.word2count = {}
      self.index2word = {0: "SOS", 1: "EOS"}
      self.n_words = 2  # Count SOS and EOS
      self.radical2index = {}
      self.radical2count = {}
      self.index2radical = {0: "SOS", 1: "EOS"}
      self.n_radical = 2  # Count SOS and EOS
      with open(self.data_dir, 'r') as myfile:
         data=myfile.read().replace('\n', '')
      self.ws = WordSegment(data, max_word_len=1, min_aggregation=10, min_entropy=0.5)
      
    def addSentence(self, sentence):
        # Chinese word segmentation
      segmented = self.ws.segSentence(sentence)
      for word in segmented:
        self.addWord(word)
        
    def addWord(self, word):
        if word not in self.word2index:
          self.word2index[word] = self.n_words
          self.word2count[word] = 1
          self.index2word[self.n_words] = word
          self.n_words += 1
        else:
          self.word2count[word] += 1
        
      

    def addword(self):
        # Chinese word segmentation
        for word in self.word2index:
          radical=find_radical(word)
          self.addradical(radical)
        

    def addradical(self,radical):
        if radical not in self.radical2index:
            self.radical2index[radical] = self.n_radical
            self.radical2count[radical] = 1
            self.index2radical[self.n_radical] = radical
            self.n_radical += 1
        else:
            self.radical2count[radical] += 1

The files are all in Unicode, to simplify we will turn Unicode
characters to ASCII, make everything lowercase, and trim most
punctuation.




In [0]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [0]:
def readLangs():
    print("Reading lines...")
    
    # Read casia2015_ch.txt
    lines_ch = open(ch_corpus_dir, encoding='utf-8').\
        read().strip().split('\n')
    
    # Read casia2015_en.txt
    lines_en = open(en_corpus_dir, encoding='utf-8').\
        read().strip().split('\n')

    # Make parallel pairs by spliting every line and normalize
    pairs = [(a, b) for a,b in zip(lines_ch, lines_en)]
    # print(pairs)
    
    # Make Lang instances
    ch_lang = ch_Lang('ch')
    en_lang = en_Lang('en')
    radical_lang =radical_Lang('ch')

    return ch_lang, en_lang, radical_lang, pairs

In [0]:
def prepareData(lang1, lang2):
    input_lang, output_lang, radical_lang, pairs = readLangs()
    print("Read %s sentence pairs" % len(pairs))
    # pairs = filterPairs(pairs)
    # print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
        radical_lang.addSentence(pair[0])
        radical_lang.addword()
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    print(radical_lang.name,radical_lang.n_words)
    return input_lang, output_lang,radical_lang, pairs


input_lang, output_lang, radical_lang, pairs = prepareData('ch', 'en')
print(random.choice(pairs))

Reading lines...
Read 100 sentence pairs
Counting words...
Counted words:
ch 1040
en 994
ch 942
('别逼着他们告诉你他们的心事。那将惹怒他们，可能另他们感觉更糟。', "Don't push them to tell you what's on their mind. That'll annoy them, and likely make them feel worse.")



```
# This is formatted as code
```

take in one chinese sentence and return a character list and a radical list

In [0]:
def get_character_radical(sentence):
    ws = WordSegment(sentence, max_word_len=1, min_aggregation=1, min_entropy=0.5)
    characters_list=ws.segSentence(sentence)
    i=0
    radical_list=[]
    while i<len(characters_list):
        radical=find_radical(characters_list[i])  
        radical_list.append(radical)
        i=i+1
    return characters_list, radical_list

In [0]:

#input:vocabulary can be a list of character/radicals
def vectorize(vocabulary):
    index=vocabulary.word2index
    lookup_tensor=torch.tensor(index[current_word], dtype=torch.long).view(-1, 1)
    word_embed.append(lookup_tensor)
    return word_embed

In [0]:
#this method take in a word and its tensor, break the word into characters and radicals and return the concatenate tensor
from wordseg import WordSegment
import numpy as np
def concatenate_wcr(word,word_tensor):
  character,radical=get_character_radical(word)
  character_tensor=vectorize(character)
  radical_tensor=vectorize(radical)
  character_sum=np.sum(character_tensor)
  radical_sum=np.sum(radical_tensor)
  ch_tensor=torch.cat((word_tensor,character_sum,radical_sum),dim=0)
  return ch_tensor
  



Preparing Training Data
-----------------------

To train, for each pair we will need an input tensor (indexes of the
words in the input sentence) and target tensor (indexes of the words in
the target sentence). While creating these vectors we will append the
EOS token to both sequences.




In [0]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


For chinese sentence

In [0]:
def indexsfromsentence(lang, sentence):
    ws=WordSegment(sentence, max_word_len=1, min_aggregation=1, min_entropy=0.5)
    characters=ws.segSentence(sentence)
    return [lang.word2index[word] for word in characters]

In [0]:
def tensorfromsentence(lang, sentence):
    indexes = indexsfromsentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [0]:
#lang is a radical_lang object
def indexsfromradical(lang, sentence):
    character, radical_list=get_character_radical(sentence)
    return [lang.radical2index[radical] for radical in radical_list]

In [0]:
def tensorfromradical(lang, sentence):
    indexes = indexsfromradical(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)

In [0]:
def tensorsFromPair(pair):
    word_tensor =tensorfromsentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    radical_tensor=tensorfromradical(radical_lang,pair[0])
    input_tensor=torch.cat((word_tensor,radical_tensor),dim=0)
    return (input_tensor, target_tensor,radical_tensor)
    #return (input_tensor,target_tensor)

In [0]:
MAX_LENGTH=150

The Seq2Seq Model
=================

A Recurrent Neural Network, or RNN, is a network that operates on a
sequence and uses its own output as input for subsequent steps.

A seq-to-seq network is a model
consisting of two RNNs called the encoder and decoder. The encoder reads
an input sequence and outputs a single vector, and the decoder reads
that vector to produce an output sequence.

Unlike sequence prediction with a single RNN, where every input
corresponds to an output, the seq2seq model frees us from sequence
length and order, which makes it ideal for translation between two
languages.

With a seq2seq model the encoder creates a single vector which, in the
ideal case, encodes the "meaning" of the input sequence into a single
vector — a single point in some N dimensional space of sentences.


The Encoder
-----------

The encoder of a seq2seq network is a RNN that outputs some value for
every word from the input sentence. For every input word the encoder
outputs a vector and a hidden state, and uses the hidden state for the
next input word.




In [0]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------

The decoder is another RNN that takes the encoder output vector(s) and
outputs a sequence of words to create the translation.




### Simple Decoder


In the simplest seq2seq decoder we use only last output of the encoder.
This last output is sometimes called the *context vector* as it encodes
context from the entire sequence. This context vector is used as the
initial hidden state of the decoder.

At every step of decoding, the decoder is given an input token and
hidden state. The initial input token is the start-of-string ``<SOS>``
token, and the first hidden state is the context vector (the encoder's
last hidden state).





In [0]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

### Attention Decoder

If only the context vector is passed betweeen the encoder and decoder,
that single vector carries the burden of encoding the entire sentence.

Attention allows the decoder network to "focus" on a different part of
the encoder's outputs for every step of the decoder's own outputs. First
we calculate a set of *attention weights*. These will be multiplied by
the encoder output vectors to create a weighted combination. The result
(called ``attn_applied`` in the code) should contain information about
that specific part of the input sequence, and thus help the decoder
choose the right output words.


Calculating the attention weights is done with another feed-forward
layer ``attn``, using the decoder's input and hidden state as inputs.
Because there are sentences of all sizes in the training data, to
actually create and train this layer we have to choose a maximum
sentence length (input length, for encoder outputs) that it can apply
to. Sentences of the maximum length will use all the attention weights,
while shorter sentences will only use the first few.






In [0]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Training the Model
------------------

To train we run the input sentence through the encoder, and keep track
of every output and the latest hidden state. Then the decoder is given
the ``<SOS>`` token as its first input, and the last hidden state of the
encoder as its first hidden state.



In [0]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

This is a helper function to print time elapsed and estimated time
remaining given the current time and progress %.




In [0]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

The whole training process looks like this:

-  Start a timer
-  Initialize optimizers and criterion
-  Create set of training pairs
-  Start empty losses array for plotting

Then we call ``train`` many times and occasionally print the progress (%
of examples, time so far, estimated time) and average loss.




In [0]:
def trainIters(encoder, decoder, n_iters, print_every=10, plot_every=100, save_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))
        if iter % save_every == 0:
            print("Encoder state dict:")
            for param_tensor in encoder.state_dict():
                print(param_tensor, "\t", encoder.state_dict()[param_tensor].size())
            print("Decoder state dict:")
            for param_tensor in decoder.state_dict():
                print(param_tensor, "\t", decoder.state_dict()[param_tensor].size())
            print("Encoder Optimizer's state dict:")
            for var_name in encoder_optimizer.state_dict():
                print(var_name, "\t", encoder_optimizer.state_dict()[var_name])
            print("Decoder Optimizer's state dict:")
            for var_name in decoder_optimizer.state_dict():
                print(var_name, "\t", decoder_optimizer.state_dict()[var_name])

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

Plotting results
----------------

Plotting is done with matplotlib, using the array of loss values
``plot_losses`` saved while training.




In [0]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

Evaluation
==========

Evaluation is mostly the same as training, but there are no targets so
we simply feed the decoder's predictions back to itself for each step.
Every time it predicts a word we add it to the output string, and if it
predicts the EOS token we stop there. We also store the decoder's
attention outputs for display later.




In [0]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        # input_tensor = tensorFromSentence(input_lang, sentence)
        input_tensor = tensorfromsentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
          
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

We can evaluate random sentences from the training set and print out the
input, target, and output to make some subjective quality judgements:




In [0]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

In [0]:
def evaluateRandomlySingle(encoder, decoder):
    pair = random.choice(pairs)
    input_sent = pair[0]
    gt = pair[1]
    output_words, attentions = evaluate(encoder, decoder, pair[0])
    output_sent = ' '.join(output_words)
    return input_sent, gt, output_sent

Training and Evaluating
=======================
   If you run this notebook you can train, interrupt the kernel,
   evaluate, and continue training later. Comment out the lines where the
   encoder and decoder are initialized and run ``trainIters`` again.




In [0]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device) 
print(encoder1)
print(attn_decoder1)
print('encoder works')
trainIters(encoder1, attn_decoder1, 75000, print_every=10)

EncoderRNN(
  (embedding): Embedding(1040, 256)
  (gru): GRU(256, 256)
)
AttnDecoderRNN(
  (embedding): Embedding(994, 256)
  (attn): Linear(in_features=512, out_features=150, bias=True)
  (attn_combine): Linear(in_features=512, out_features=256, bias=True)
  (dropout): Dropout(p=0.1)
  (gru): GRU(256, 256)
  (out): Linear(in_features=256, out_features=994, bias=True)
)
encoder works


KeyError: ignored

In [0]:
# Save model
# torch.save(encoder1.state_dict(), 'models/encoder1.txt')
# torch.save(attn_decoder1.state_dict(), 'models/attn_decoder1.txt')

In [0]:
evaluateRandomly(encoder1, attn_decoder1)

In [0]:
print(evaluateRandomlySingle(encoder1, attn_decoder1))

Blingual Evaluation Understudy Score

In [0]:
from nltk.translate import bleu
from nltk.translate.bleu_score import SmoothingFunction
scores = []
func = SmoothingFunction().method4
for i in range(1000):
    chinese, gt, english = evaluateRandomlySingle(encoder1,attn_decoder1)
    smoothing_scores = bleu(gt, english, smoothing_function=func)
    scores.append(smoothing_scores)
print(np.mean(scores))